## Computational Models
---
+ Optimization models
+ Statistical models
+ Simulation models

## what is an Optimization Model?
---
**1. An objective funcion that is to be maximized or minimized**  

**2. A set of constraints(possibly empty) that must be honored**

## Knapsack Problem (背包问题）

### Brute Force Algorithm
1. Enumerate all possible combinations of items. That is to say, generate all subsets of the set of items. This is
called the **power set**.
2. Remove all of the combinations whose total units exceeds the allowd weight.
3. From the remaining combinaitons choose any one whose value is the largest.

## Greedy Algorithm a Practical Alternative
---
+ while knapsack not full: put best available item in knapsack
+ But what does best mean?
   + Most valuable
   + Lest expensive
   + Higest value/units

In [7]:
class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w
    
    def getValue(self):
        return self.value
    
    def getCost(self):
        return self.calories
    
    def density(self):
        return self.getValue()/self.getCost()
    
    def __str__(self):
        return self.name + ":<" + str(self.value) + "," + str(self.calories) + ">"
    

def buildMenu(names, values, calories):
    '''
    names,values,calories lists of same length.
    name a  list of strings
    values and calories list of numbers
    return: list of foods
    '''
    menu = []
    for i in range(len(values)):
        menu.append(Food(names[i],values[i],calories[i]))
    return menu

In [11]:
def greedy(items, maxCost, keyFunction):
    """Assumes items a list, maxCost >= 0,
         keyFunction maps elements of items to numbers"""
    itemsCopy = sorted(items, key = keyFunction, reverse = True) # keyFunction :type == function
    result = []
    totalValue, totalCost = 0.0, 0.0
    for i in range(len(itemsCopy)):
        if (totalCost+itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalCost += itemsCopy[i].getCost()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)

## sorted(iterable, key=None, reverse=False)
+ iterable -- 可迭代对象
+ key -- 用来进行比较的对象，具体的函数的参数就是取自于可迭代对象中，指定可迭代对象的一个元素进行排序
+ reverse -- true：降序；false：升序
+ return：重新排序的列表

In [17]:
def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print('   ', item)

def testGreedys(foods, maxUnits):
    print('Use greedy by value to allocate', maxUnits, 'calories')
    testGreedy(foods, maxUnits, Food.getValue)
    print('\nUse greedy by cost to allocate', maxUnits, 'calories')
    testGreedy(foods, maxUnits, lambda x: 1/Food.getCost(x))
    print('\nUse greedy by density to allocate', maxUnits, 'calories')
    testGreedy(foods, maxUnits, Food.density)

## lambda
### lambda used to create anonymous functions 匿名函数
+ lambda < id1,id2,... idn>: expression
+ returns a funciton of n arguments
---
can be very handy  
possible to write amazing complicated lambda expressions

Example:
```
f = lambda x,y: x**2 + y**2
f(3,4) == 25
```

In [15]:
names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)
testGreedys(foods, 1000)

Use greedy by value to allocate 1000 calories
Total value of items taken = 424.0
    burger:<100,354>
    pizza:<95,258>
    beer:<90,154>
    wine:<89,123>
    apple:<50,95>

Use greedy by cost to allocate 1000 calories
Total value of items taken = 413.0
    apple:<50,95>
    wine:<89,123>
    cola:<79,150>
    beer:<90,154>
    donut:<10,195>
    pizza:<95,258>

Use greedy by density to allocate 1000 calories
Total value of items taken = 413.0
    wine:<89,123>
    beer:<90,154>
    cola:<79,150>
    apple:<50,95>
    pizza:<95,258>
    donut:<10,195>


## why different answers?
**Sequence fo locally optimal choices don't always yield a globally optimal solution**
## The Pros and Cos of Greedy
**Easy to implement**  
**Computationally efficient**  
**But does not always yield the best solution**  
   Don't even know how good the approximation is

# Brute Force Algorithm
## Search Tree Implementation
1. The tree is built top down starting with the cost
2. The first element is selected from the still to be considered items
3. The process is then applied recursively to non-leaf children
4. Finally, choose a node with the highest value that meets constraints

![avatar](./img/SearchTree.png)

## Computational Complexity
1. Time based on number of nodes generated
2. Number of levels is number of items to choose from
3. Number of nodes at level i is 2^i
4. if there are n items, the number of nodes is 2^(n+1). O(2^n)
5. An obvious optimization: don't explore parts of tree  that violate constraint
**Doesn't change complexity**

## Header of Decision Tree Implementation

In [18]:
def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake = maxVal(toConsider[1:],
                                     avail - nextItem.getCost())
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

In [19]:
def testMaxVal(foods, maxUnits, printItems = True):
    print('Use search tree to allocate', maxUnits,
          'calories')
    val, taken = maxVal(foods, maxUnits)
    print('Total value of items taken =', val)
    if printItems:
        for item in taken:
            print('   ', item)

names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10]
calories = [123,154,258,354,365,150,95,195]
foods = buildMenu(names, values, calories)

testGreedys(foods, 750)
print('')
testMaxVal(foods, 750)

Use greedy by value to allocate 750 calories
Total value of items taken = 284.0
    burger:<100,354>
    pizza:<95,258>
    wine:<89,123>

Use greedy by cost to allocate 750 calories
Total value of items taken = 318.0
    apple:<50,95>
    wine:<89,123>
    cola:<79,150>
    beer:<90,154>
    donut:<10,195>

Use greedy by density to allocate 750 calories
Total value of items taken = 318.0
    wine:<89,123>
    beer:<90,154>
    cola:<79,150>
    apple:<50,95>
    donut:<10,195>

Use search tree to allocate 750 calories
Total value of items taken = 353
    cola:<79,150>
    pizza:<95,258>
    beer:<90,154>
    wine:<89,123>


## Code to Try Larger Examples

In [20]:
import random

def buildLargeMenu(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Food(str(i), random.randint(1, maxVal), random.randint(1, maxCost)))
    return items

for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45):
    print('Try a menu with', numItems, 'items')
    items = buildLargeMenu(numItems, 90, 250)
    testMaxVal(items, 750, False)  

Try a menu with 5 items
Use search tree to allocate 750 calories
Total value of items taken = 292
Try a menu with 10 items
Use search tree to allocate 750 calories
Total value of items taken = 317
Try a menu with 15 items
Use search tree to allocate 750 calories
Total value of items taken = 571
Try a menu with 20 items
Use search tree to allocate 750 calories
Total value of items taken = 505
Try a menu with 25 items
Use search tree to allocate 750 calories
Total value of items taken = 456
Try a menu with 30 items
Use search tree to allocate 750 calories
Total value of items taken = 533
Try a menu with 35 items
Use search tree to allocate 750 calories
Total value of items taken = 812
Try a menu with 40 items
Use search tree to allocate 750 calories
Total value of items taken = 804
Try a menu with 45 items
Use search tree to allocate 750 calories
Total value of items taken = 688


## Dynamic Programming

### Recursive Implementation of Fibonnaci

In [21]:
def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

### Call Tree for Recursive Fibonnaci(6) = 13

![calltree](./img/CallTree.png)

## Clearly a Bad Idea to Repeat Work
1. Trade a time for space
2. Create a table to record what we've done
3. called memoization

In [22]:
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
       Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

In [24]:
for i in range(100,121):
    print('fib(' + str(i) + ') =', fastFib(i))

fib(100) = 573147844013817084101
fib(101) = 927372692193078999176
fib(102) = 1500520536206896083277
fib(103) = 2427893228399975082453
fib(104) = 3928413764606871165730
fib(105) = 6356306993006846248183
fib(106) = 10284720757613717413913
fib(107) = 16641027750620563662096
fib(108) = 26925748508234281076009
fib(109) = 43566776258854844738105
fib(110) = 70492524767089125814114
fib(111) = 114059301025943970552219
fib(112) = 184551825793033096366333
fib(113) = 298611126818977066918552
fib(114) = 483162952612010163284885
fib(115) = 781774079430987230203437
fib(116) = 1264937032042997393488322
fib(117) = 2046711111473984623691759
fib(118) = 3311648143516982017180081
fib(119) = 5358359254990966640871840
fib(120) = 8670007398507948658051921


## When Dose It Work?
### Optimal substructure: a globally optimal solution can be found by combining optimal solutions to local subproblems
### Overlapping subproblems: finding an optimal solution involves solving the same problem multiple times

## Modify maxVal to Use a Memo

In [25]:
def fastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of subjects, avail a weight
         memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the
         0/1 knapsack problem and the subjects of that solution"""
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        #Explore right branch only
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 fastMaxVal(toConsider[1:],
                            avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result


In [28]:
def testMaxVal(foods, maxUnits, algorithm, printItems = True):
    print('Menu contains', len(foods), 'items')
    print('Use search tree to allocate', maxUnits, 'calories')
    val, taken = algorithm(foods, maxUnits)
    if printItems:
        print('Total value of items taken =', val)
        for item in taken:
            print('   ', item)
            
            
for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45, 50):
    items = buildLargeMenu(numItems, 90, 250)
    testMaxVal(items, 750, fastMaxVal, True)

Menu contains 5 items
Use search tree to allocate 750 calories
Total value of items taken = 221
    4:<50,161>
    3:<58,131>
    2:<50,171>
    1:<63,175>
Menu contains 10 items
Use search tree to allocate 750 calories
Total value of items taken = 282
    9:<55,176>
    7:<10,39>
    6:<35,97>
    4:<25,2>
    3:<83,101>
    2:<55,221>
    1:<19,18>
Menu contains 15 items
Use search tree to allocate 750 calories
Total value of items taken = 499
    14:<78,64>
    12:<44,2>
    11:<21,100>
    10:<42,66>
    9:<89,127>
    7:<55,65>
    5:<77,199>
    3:<52,4>
    2:<41,41>
Menu contains 20 items
Use search tree to allocate 750 calories
Total value of items taken = 565
    14:<78,64>
    12:<44,2>
    10:<42,66>
    9:<89,127>
    8:<55,132>
    7:<55,65>
    4:<21,97>
    3:<52,4>
    7:<61,88>
    4:<68,95>
Menu contains 25 items
Use search tree to allocate 750 calories
Total value of items taken = 645
    12:<44,2>
    10:<42,66>
    9:<89,127>
    18:<74,80>
    17:<87,30>
    13:<

# Summary
### Many problems of parctical importance can be formulated as optimization problems
### Greedy algorithms often provide adequent(though not necessarily optimal) solutions
### Finding an optimal solution is usually exponentially hard
### But dynamic programming often yields good performance for a subclass of optimization problems--- those with optimal substructure and overlapping subproblems
---
1. **Solution always correct**
2. **Fast under the right circumstances**